In [1]:
import geopandas as gpd
import pandas as pd

Read data

In [2]:
cook = gpd.read_file('data/cook_tract.shp')
cook_block = gpd.read_file('data/cook_block.shp')
acs = pd.read_csv('data/nhgis0007_ds263_20225_tract.csv', encoding='latin-1')
pop = pd.read_csv('data/nhgis0008_ds258_2020_block.csv', encoding='latin-1')

C:\Users\Yue\AppData\Local\Temp\ipykernel_9564\957685708.py:4: DtypeWarning: Columns (40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  pop = pd.read_csv('data/nhgis0008_ds258_2020_block.csv', encoding='latin-1')


Population-weighted centroids

In [58]:
cook_block = cook_block.merge(pop[['GISJOIN', 'U7H001']], on='GISJOIN')

# Calculate the population-weighted center for each tract in cook_tract
tract_centers = []
for _, tract in cook.iterrows():
    blocks_in_tract = cook_block[cook_block['geometry'].within(tract['geometry'])]
    
    weighted_center = blocks_in_tract['geometry'].centroid
    weighted_population = blocks_in_tract['U7H001']

    total_weighted_population = weighted_population.sum()

    if total_weighted_population > 0:
        # Use population-weighted centroid if population is greater than 0
        weighted_longitude = (weighted_center.x * weighted_population).sum() / total_weighted_population
        weighted_latitude = (weighted_center.y * weighted_population).sum() / total_weighted_population
    else:
        # Use geometric centroid if population is 0
        weighted_longitude = tract['geometry'].centroid.x
        weighted_latitude = tract['geometry'].centroid.y

    tract_centers.append({
        'GISJOIN': tract['GISJOIN'],
        'Lon_w': weighted_longitude,
        'Lat_w': weighted_latitude
    })

tract_centers_gdf = gpd.GeoDataFrame(tract_centers, geometry=gpd.points_from_xy([center['Lon_w'] for center in tract_centers], [center['Lat_w'] for center in tract_centers]))
cook_centroids = tract_centers_gdf.merge(cook, on='GISJOIN')
cook_centroids = cook_centroids.drop('geometry_y', axis=1)
cook_centroids = cook_centroids.rename(columns={'geometry_x': 'geometry'})
cook_centroids = cook_centroids.set_geometry('geometry')
cook_centroids.crs = cook.crs
cook_centroids

,GISJOIN,Lon_w,Lat_w,geometry,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,Shape_Leng,Shape_Area,ORIG_FID
0,G1700310010100,684221.785904,536506.549117,POINT (684221.786 536506.549),17,031,010100,17031010100,101,Census Tract 101,G5020,S,379511.0,0.0,+42.0212553,-087.6698301,2833.024949,3.795106e+05,39803
1,G1700310010201,683361.585376,535848.771305,POINT (683361.585 535848.771),17,031,010201,17031010201,102.01,Census Tract 102.01,G5020,S,504464.0,0.0,+42.0160077,-087.6801484,2837.325814,5.044621e+05,39016
2,G1700310010202,684032.517605,535872.761074,POINT (684032.518 535872.761),17,031,010202,17031010202,102.02,Census Tract 102.02,G5020,S,351253.0,0.0,+42.0160476,-087.6733256,2497.850009,3.512543e+05,40224
3,G1700310010300,684542.829846,535948.251783,POINT (684542.830 535948.252),17,031,010300,17031010300,103,Census Tract 103,G5020,S,472522.0,0.0,+42.0159431,-087.6665393,2972.474400,4.725243e+05,39699
4,G1700310010400,685225.899687,534657.507557,POINT (685225.900 534657.508),17,031,010400,17031010400,104,Census Tract 104,G5020,S,548761.0,602605.0,+42.0064109,-087.6588157,4351.049600,5.487605e+05,39700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,G1700310843900,694302.110504,509607.030347,POINT (694302.111 509607.030),17,031,843900,17031843900,8439,Census Tract 8439,G5020,S,1478136.0,729136.0,+41.7789932,-087.5761303,7172.016080,1.625594e+06,40454
1327,G1700310844600,690000.312414,513706.489801,POINT (690000.312 513706.490),17,031,844600,17031844600,8446,Census Tract 8446,G5020,S,434671.0,0.0,+41.8133852,-087.6254437,2897.497712,4.346710e+05,38684
1328,G1700310844700,682417.843484,517566.140090,POINT (682417.843 517566.140),17,031,844700,17031844700,8447,Census Tract 8447,G5020,S,401852.0,0.0,+41.8541473,-087.7117002,3340.233009,4.018555e+05,38683
1329,G1700310980000,665423.171153,530281.953343,POINT (665423.171 530281.953),17,031,980000,17031980000,9800,Census Tract 9800,G5020,S,19890204.0,92402.0,+41.9794191,-087.9024376,21461.280911,1.998261e+07,39747


Add attributes

In [59]:
cook_centroids = cook_centroids.merge(acs, on='GISJOIN')

# sex
cook_centroids['S1'] = cook_centroids[['AQYHE002', 'AQYIE002', 'AQYJE002', 'AQYKE002', 'AQYLE002', 'AQYME002', 'AQYNE002']].sum(axis=1)
cook_centroids['S2'] = cook_centroids[['AQYHE017', 'AQYIE017', 'AQYJE017', 'AQYKE017', 'AQYLE017', 'AQYME017', 'AQYNE017']].sum(axis=1)

# age
columns_A1 = [f'AQY{letter}E{num}' for letter in 'HIJKLMN' for num in ['003', '004', '005', '018', '019', '020']]
cook_centroids['A1'] = cook_centroids[columns_A1].sum(axis=1)
columns_A2 = [f'AQY{letter}E{num}' for letter in 'HIJKLMN' for num in ['006', '007', '008', '009', '010', '011', '012', '013', '021', '022', '023', '024', '025', '026', '027', '028']]
cook_centroids['A2'] = cook_centroids[columns_A2].sum(axis=1)
columns_A3 = [f'AQY{letter}E{num}' for letter in 'HIJKLMN' for num in ['014', '015', '016', '029', '030', '031']]
cook_centroids['A3'] = cook_centroids[columns_A3].sum(axis=1)

# race
suffixes = ['H', 'I', 'J', 'K', 'L', 'M', 'N']
for i, suffix in enumerate(suffixes, start=1):
    cook_centroids[f'R{i}'] = cook_centroids[f'AQY{suffix}E001']

# income
columns_HHI1 = [f'AQ7{letter}E{num}' for letter in 'YZ01234' for num in ['002', '003', '004', '005', '006', '007', '008', '009']]
cook_centroids['HHI1'] = cook_centroids[columns_HHI1].sum(axis=1)
columns_HHI2 = [f'AQ7{letter}E{num}' for letter in 'YZ01234' for num in ['010', '011', '012', '013', '014', '015']]
cook_centroids['HHI2'] = cook_centroids[columns_HHI2].sum(axis=1)
columns_HHI3 = [f'AQ7{letter}E{num}' for letter in 'YZ01234' for num in ['016', '017']]
cook_centroids['HHI3'] = cook_centroids[columns_HHI3].sum(axis=1)

# total
cook_centroids['Total_Pop'] = cook_centroids[[f'AQY{letter}E001' for letter in 'HIJKLMN']].sum(axis=1)
cook_centroids['Total_HH'] = cook_centroids[[f'AQ7{letter}E001' for letter in 'YZ01234']].sum(axis=1)

cook_centroids = cook_centroids.drop(cook_centroids.filter(like='AQ').columns, axis=1)
cook_centroids.to_file('data/cook_centroids_all.shp')
cook_centroids

,GISJOIN,Lon_w,Lat_w,geometry,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,...,R3,R4,R5,R6,R7,HHI1,HHI2,HHI3,Total_Pop,Total_HH
0,G1700310010100,684221.785904,536506.549117,POINT (684221.786 536506.549),17,031,010100,17031010100,101,Census Tract 101,...,0,94,0,278,621,818,1145,359,4284,2322
1,G1700310010201,683361.585376,535848.771305,POINT (683361.585 535848.771),17,031,010201,17031010201,102.01,Census Tract 102.01,...,41,298,0,838,1937,1363,1461,204,8210,3028
2,G1700310010202,684032.517605,535872.761074,POINT (684032.518 535872.761),17,031,010202,17031010202,102.02,Census Tract 102.02,...,86,312,0,353,277,469,646,108,2865,1223
3,G1700310010300,684542.829846,535948.251783,POINT (684542.830 535948.252),17,031,010300,17031010300,103,Census Tract 103,...,76,338,0,704,303,1040,1560,447,6237,3047
4,G1700310010400,685225.899687,534657.507557,POINT (685225.900 534657.508),17,031,010400,17031010400,104,Census Tract 104,...,0,213,0,71,257,860,1065,133,4473,2058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,G1700310843900,694302.110504,509607.030347,POINT (694302.111 509607.030),17,031,843900,17031843900,8439,Census Tract 8439,...,29,50,0,98,73,1017,1130,54,3700,2201
1327,G1700310844600,690000.312414,513706.489801,POINT (690000.312 513706.490),17,031,844600,17031844600,8446,Census Tract 8446,...,0,0,24,15,80,300,445,83,2275,828
1328,G1700310844700,682417.843484,517566.140090,POINT (682417.843 517566.140),17,031,844700,17031844700,8447,Census Tract 8447,...,0,0,0,156,151,310,308,29,1753,647
1329,G1700310980000,665423.171153,530281.953343,POINT (665423.171 530281.953),17,031,980000,17031980000,9800,Census Tract 9800,...,0,0,0,0,0,0,0,0,0,0


Tract data (for visual)

In [3]:
cook = cook.merge(acs, on='GISJOIN')

# sex
cook['S1'] = cook[['AQYHE002', 'AQYIE002', 'AQYJE002', 'AQYKE002', 'AQYLE002', 'AQYME002', 'AQYNE002']].sum(axis=1)
cook['S2'] = cook[['AQYHE017', 'AQYIE017', 'AQYJE017', 'AQYKE017', 'AQYLE017', 'AQYME017', 'AQYNE017']].sum(axis=1)

# age
columns_A1 = [f'AQY{letter}E{num}' for letter in 'HIJKLMN' for num in ['003', '004', '005', '018', '019', '020']]
cook['A1'] = cook[columns_A1].sum(axis=1)
columns_A2 = [f'AQY{letter}E{num}' for letter in 'HIJKLMN' for num in ['006', '007', '008', '009', '010', '011', '012', '013', '021', '022', '023', '024', '025', '026', '027', '028']]
cook['A2'] = cook[columns_A2].sum(axis=1)
columns_A3 = [f'AQY{letter}E{num}' for letter in 'HIJKLMN' for num in ['014', '015', '016', '029', '030', '031']]
cook['A3'] = cook[columns_A3].sum(axis=1)

# race
suffixes = ['H', 'I', 'J', 'K', 'L', 'M', 'N']
for i, suffix in enumerate(suffixes, start=1):
    cook[f'R{i}'] = cook[f'AQY{suffix}E001']

# income
columns_HHI1 = [f'AQ7{letter}E{num}' for letter in 'YZ01234' for num in ['002', '003', '004', '005', '006', '007', '008', '009']]
cook['HHI1'] = cook[columns_HHI1].sum(axis=1)
columns_HHI2 = [f'AQ7{letter}E{num}' for letter in 'YZ01234' for num in ['010', '011', '012', '013', '014', '015']]
cook['HHI2'] = cook[columns_HHI2].sum(axis=1)
columns_HHI3 = [f'AQ7{letter}E{num}' for letter in 'YZ01234' for num in ['016', '017']]
cook['HHI3'] = cook[columns_HHI3].sum(axis=1)

# total
cook['Total_Pop'] = cook[[f'AQY{letter}E001' for letter in 'HIJKLMN']].sum(axis=1)
cook['Total_HH'] = cook[[f'AQ7{letter}E001' for letter in 'YZ01234']].sum(axis=1)

cook = cook.drop(cook.filter(like='AQ').columns, axis=1)
cook.to_file('data/cook_tract_all.shp')
cook

,GISJOIN,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,R3,R4,R5,R6,R7,HHI1,HHI2,HHI3,Total_Pop,Total_HH
0,G1700310010100,17,031,010100,17031010100,101,Census Tract 101,G5020,S,379511.0,...,0,94,0,278,621,818,1145,359,4284,2322
1,G1700310010201,17,031,010201,17031010201,102.01,Census Tract 102.01,G5020,S,504464.0,...,41,298,0,838,1937,1363,1461,204,8210,3028
2,G1700310010202,17,031,010202,17031010202,102.02,Census Tract 102.02,G5020,S,351253.0,...,86,312,0,353,277,469,646,108,2865,1223
3,G1700310010300,17,031,010300,17031010300,103,Census Tract 103,G5020,S,472522.0,...,76,338,0,704,303,1040,1560,447,6237,3047
4,G1700310010400,17,031,010400,17031010400,104,Census Tract 104,G5020,S,548761.0,...,0,213,0,71,257,860,1065,133,4473,2058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,G1700310843900,17,031,843900,17031843900,8439,Census Tract 8439,G5020,S,1478136.0,...,29,50,0,98,73,1017,1130,54,3700,2201
1327,G1700310844600,17,031,844600,17031844600,8446,Census Tract 8446,G5020,S,434671.0,...,0,0,24,15,80,300,445,83,2275,828
1328,G1700310844700,17,031,844700,17031844700,8447,Census Tract 8447,G5020,S,401852.0,...,0,0,0,156,151,310,308,29,1753,647
1329,G1700310980000,17,031,980000,17031980000,9800,Census Tract 9800,G5020,S,19890204.0,...,0,0,0,0,0,0,0,0,0,0
